In [128]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from uncertainties import ufloat
from uncertainties.umath import *  # sin(), etc.
import uncertainties.unumpy as un

df_placa = pd.read_excel('D:/UNESP/EngMec-UNESP/Lab. Mec Flu II/Lab 1 - Medidores de vazão/dados.xlsx')
df_vent = pd.read_excel('D:/UNESP/EngMec-UNESP/Lab. Mec Flu II/Lab 1 - Medidores de vazão/dados.xlsx', sheet_name='venturi')

In [129]:
rho_ar =1.20
rho_agua = 997
mi = 1.8e-5
g = 9.81

def calc_vazão(Dt, D1, delta_h):
    
    beta = Dt/D1
    At = np.pi*(Dt**2)/4
    A1 = np.pi*(D1**2)/4

    delta_P = rho_agua*g*(delta_h/100)
    m_dot  = 1.0

    lista_m = []
    for j in range(len(delta_P)):
        for i in range(100):
            v = m_dot/(rho_ar*A1)
            Re = (rho_ar*v*D1)/mi
            C = 0.5959 + 0.0312*(beta**2.1) - 0.184*beta**8 + (91.71*beta**2.5)/(Re**0.75)
            K = C/np.sqrt(1-beta**4)
            m_dot_new = K*At*np.sqrt(2*rho_ar*(delta_P[j]))
            
            if abs(m_dot - m_dot_new) < 1e-8:
                break
            else:
                m_dot = m_dot_new

        lista_m.append(m_dot_new)
        
    lista_m = np.array(lista_m)
    lista_Q = lista_m/rho_ar
    lista_V = lista_Q/A1

    return lista_m, lista_Q, lista_V #kg/s, m^3/s, m/s

In [130]:
#Placa de orificio
placa = calc_vazão(0.0257, 0.0385, df_placa['del'].values)
placa

(array([0.00768234, 0.01389186, 0.01812435, 0.02062295, 0.0242119 ]),
 array([0.00640195, 0.01157655, 0.01510362, 0.0171858 , 0.02017659]),
 array([ 5.49921472,  9.9441543 , 12.97387615, 14.76244499, 17.33150751]))

In [131]:
res_p = pd.DataFrame({'teo':placa[2], 'exp': df_placa['ve'].values, 
                     'erro':abs(df_placa['ve'].values - placa[2])/placa[2]*100})
res_p

,teo,exp,erro
0,5.499215,3.40,38.172991
1,9.944154,5.94,40.266414
2,12.973876,7.65,41.035355
3,14.762445,8.60,41.744067
4,17.331508,9.93,42.705503


In [132]:
#Tubo de venturi
tubo = calc_vazão(0.0257, 0.0385, df_vent['del'].values)
tubo

(array([0.0061508 , 0.01130399, 0.01559366, 0.01883936, 0.02510037]),
 array([0.00512567, 0.00942   , 0.01299472, 0.01569946, 0.02091697]),
 array([ 4.40290493,  8.09169203, 11.16234557, 13.48569917, 17.96749481]))

In [133]:
res_t = pd.DataFrame({'teo':tubo[2], 'exp': df_vent['ve'].values, 
                     'erro':abs(df_vent['ve'].values - tubo[2])/tubo[2]*100})
res_t

,teo,exp,erro
0,4.402905,3.60,18.235800
1,8.091692,6.40,20.906530
2,11.162346,8.80,21.163523
3,13.485699,10.58,21.546522
4,17.967495,13.00,27.647120


In [134]:
res_t = pd.DataFrame({'placa':placa[2], 'tubo': tubo[2], 
                     'erro':abs(tubo[2] - placa[2])/placa[2]*100})
res_t

,placa,tubo,erro
0,5.499215,4.402905,19.935752
1,9.944154,8.091692,18.628656
2,12.973876,11.162346,13.962909
3,14.762445,13.485699,8.648607
4,17.331508,17.967495,3.669544
